In [1]:
import serial
import string
import random
import time

def generate_random_string(length):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))


def connect(port, baudrate, bytesize, parity, stopbits, xonxoff, rtscts, cnt, data_len, timeout, read=True):
    try:
        ser = serial.Serial(port, baudrate, bytesize=bytesize, parity=parity, stopbits=stopbits, timeout=timeout, write_timeout=timeout, xonxoff=xonxoff, rtscts=rtscts)
        # Write data to the serial port
        data_to_send = (chr(cnt) + generate_random_string(data_len-1)).encode('utf-8')
        
        print(f"Sent data: {data_to_send}, len {len(data_to_send)}")
        ser.write(data_to_send)

        if(read):
            # Read the same number of characters from the serial port
            data_raw = ser.read(data_len)
            data_received = data_raw.decode('utf-8')

            print(f"Readed: {data_received}")
            if len(data_received) > 0:
                print(f"Received data: {data_received}")
                if data_raw == data_to_send:
                    print("Received data matches sent data.")
                    return True
                else:
                    print("Received data does not match sent data.")
            else:
                print("No data received within the read timeout.")

        ser.close()
    except serial.SerialException as e:
        print("Error during connecting: ", e)
    return False

In [2]:
baudrates = [1200, 2400, 4800, 9600, 19200, 38400, 57600, 115200]
bytesizes = [serial.EIGHTBITS] #[serial.FIVEBITS, serial.SIXBITS, serial.SEVENBITS, serial.EIGHTBITS]
parities = [serial.PARITY_NONE, serial.PARITY_EVEN, serial.PARITY_ODD, serial.PARITY_MARK, serial.PARITY_SPACE]
stop_bits = [serial.STOPBITS_ONE,  serial.STOPBITS_TWO] #serial.STOPBITS_ONE_POINT_FIVE (druhy)
flow_controls = [None, 'RTS/CTS', 'XON/XOFF']



port = "COM5"
cnt = 0
fail = 0
for baudrate in baudrates:
    for bytesize in bytesizes:
        for parity in parities:
            for stopbits in stop_bits:
                for flow_control in flow_controls:
                    cnt += 1;
                    #if(cnt<3): continue;
                    #if(flow_control == "RTS/CTS"): continue;
                    
                    time.sleep(0.2)
                    print(f"{cnt}) Trying combination: baudrate={baudrate}, bytesize={bytesize}, parity={parity}, stopbits={stopbits}, flow_control={flow_control}")

                    xonxoff = rtscts = False
                    if flow_control == 'RTS/CTS':
                        rtscts = True
                    elif flow_control == 'XON/XOFF':
                        xonxoff = True

                    try:
                        if(False == connect(port, baudrate, bytesize, parity, stopbits, xonxoff, rtscts, cnt, 100, 5)): fail += 1
                    except Exception as e:
                        print(f"Failed with combination: baudrate={baudrate}, bytesize={bytesize}, parity={parity}, stopbits={stopbits}, flow_control={flow_control}")
                        print(f"Error: {e}")
                        fail += 1
                    print()
print(fail, "/", cnt)

1) Trying combination: baudrate=1200, bytesize=8, parity=N, stopbits=1, flow_control=None
Sent data: rsjkjVmS9hRNtsIvrji462eUpiIcjjD9Ga4269cXs1AvCXJghaYJz5puwW6iSUH64SXTpAaNTOJgGIXrxClXO2eRL7EXUWpMQMV, len 100
Readed: rsjkjVmS9hRNtsIvrji462eUpiIcjjD9Ga4269cXs1AvCXJghaYJz5puwW6iSUH64SXTpAaNTOJgGIXrxClXO2eRL7EXUWpMQMV
Received data: rsjkjVmS9hRNtsIvrji462eUpiIcjjD9Ga4269cXs1AvCXJghaYJz5puwW6iSUH64SXTpAaNTOJgGIXrxClXO2eRL7EXUWpMQMV
Received data matches sent data.

2) Trying combination: baudrate=1200, bytesize=8, parity=N, stopbits=1, flow_control=RTS/CTS
Sent data: JvO4cgR0bdd3vABFJPeh13jBDUAprzCp1eJzmImb1RMz2nCChUYWF7n1YAjTwLhsbn5hEksnrTYEvv1DzhOba0GDHaKdjv5WapS, len 100
Readed: JvO4cgR0bdd3vABFJPeh13jBDUAprzCp1eJzmImb1RMz2nCChUYWF7n1YAjTwLhsbn5hEksnrTYEvv1DzhOba0GDHaKdjv5WapS
Received data: JvO4cgR0bdd3vABFJPeh13jBDUAprzCp1eJzmImb1RMz2nCChUYWF7n1YAjTwLhsbn5hEksnrTYEvv1DzhOba0GDHaKdjv5WapS
Received data matches sent data.

3) Trying combination: baudrate=1200, bytesize=8, parity=

KeyboardInterrupt: 

# Test velkými daty / jednorázový

In [2]:
flow_control = None
xonxoff = rtscts = False
if flow_control == 'RTS/CTS':
    rtscts = True
elif flow_control == 'XON/XOFF':
    xonxoff = True

data_len = 1000
timeout = 100
connect("COM5", 19200, 8, serial.PARITY_EVEN, serial.STOPBITS_ONE_POINT_FIVE, xonxoff, rtscts, 42, data_len, timeout, True)

Sent data: b'*UoVelddxoDgKQIsvSue5j3yFgH9TjiNzQlBsU2OuqgKgNYPEuhnm6zD67MUbqdEchWm06zGKmqmxsK4o57AvraT1wfAM14Pspguzn0M1V39scVhBtuoBHS5kLZ4OfeSey332dRl92l9uHBjqbpBeeoRb4eVMt6UdKXzeji37wVFnJz2NyFrB3AWRjbJE8yH7du4vWcJMdCvbWtXN1MIivdCdVPMiWCC33Bbw5Btg5Pg0hElpqvvgduYlJXghE57qv71oEm8Wdh3cdXHXTGdWAtboZ4CPPhm5Tf182Ewm9HTDTqwgajrOt6s5yS5zxAwhchfsp8hfG37XgR1QrS8N571wSLrTyRnwRC4YcPilPi0Zc03yURtJzIiKUJea3Mb1vMy5l8sBKhFmucJ3kEZ6Y6dU6EFliMxGqownszqydX60F4aiHZ7p1s1fhNdP86sgmTmDehRqwxhChoNgQynfRSPfhKtjhKLdak6mdRju6sn0LCqTFLWEIsFAWhXm1TW7v9yDZZOgXLOk9hPyQfiCTmX4YazGC53Ak8sApdurUafC59TtdAH1LCLBfi8dytMMZjvxvFQvs6VuXBQHvUxgbzC3mBZpCkBnpujOmNx1od68W3advUBw1EwIb9MhwqOYXtkk7zaRTbo61wqgcBMxyPZrRYKtsrvGou0O7oYH5rM8A13gBiWjYIdoYeBvmoDNGkefng6cK3TqGAsmi5kvaOKAvMt6q1nhfElvmrqaZycTqQxKbh50HukOTgYE6kFmRoUSUhfAwEL52daAcfzi9GxHMWWHxdb9io1AEFtoQ3I8x9NGzhNbvUXeDJNFyHFKLxgNIjTCx8KeMTt7rN0Ry9MxJsabzszBBHGIpcBego1KrvAAcgc2vsQWlrjqwaKoc6o4KMCNvO8U6dOqr04ToTTuDjlhuGsQGd4R9cQeqvvvxwO42f6PqDfhhqlKHbEEh4Hta7umsWtIjZynseP4u6C4vs

True